# Peer-graded Assignment: Analyzing Historical Stock/Revenue Data and Building a Dashboard

In [1]:
!pip install yfinance pandas requests beautifulsoup4 plotly

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.6 MB 991.0 kB/s eta 0:00:10
    --------------------------------------- 0.2/9.6 MB 2.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.5/9.6 MB 3.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/9.6 MB 4.7 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/9.6 MB 4.7 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/9.6 MB 4.2 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.6 MB 5.1 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.6 MB 5.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.3/9.6 MB 5.3 MB/s eta 0:00:02
   ----------- ---------------------------- 2.7/9.6 MB 5.7 MB/s eta 0:00:02
   ----------- ---------------------------- 2.7/9.6 MB 5.3 MB/s eta 0:00:02
   ------------- -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\chand\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Question 1: Extracting Tesla Stock Data Using yfinance

In [3]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2: Extracting Tesla Revenue Data Using Webscraping

In [10]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")
tables = soup.find_all("table")
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in tables[1].find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text
        revenue = cols[1].text
        tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(",", "").str.replace("$", "")
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].astype("int64")
tesla_revenue["Date"] = pd.to_datetime(tesla_revenue["Date"])
tesla_revenue.tail()

IndexError: list index out of range

## Question 3: Extracting GameStop Stock Data Using yfinance

In [6]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4: Extracting GameStop Revenue Data Using Webscraping

In [7]:
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")
tables = soup.find_all("table")
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in tables[1].find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text
        revenue = cols[1].text
        gme_revenue = gme_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(",", "").str.replace("$", "")
gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]
gme_revenue["Revenue"] = gme_revenue["Revenue"].astype("int64")
gme_revenue["Date"] = pd.to_datetime(gme_revenue["Date"])
gme_revenue.tail()

IndexError: list index out of range

## Questions 5 & 6: Visualizing Tesla and GameStop Stock vs Revenue

In [8]:
def make_graph(stock_data, revenue_data, company):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=stock_data["Date"], y=stock_data["Close"], name="Stock Price"), secondary_y=False)
    fig.add_trace(go.Scatter(x=revenue_data["Date"], y=revenue_data["Revenue"], name="Revenue"), secondary_y=True)
    fig.update_layout(title=f"{company} Stock Price vs Revenue", xaxis_title="Date")
    fig.update_yaxes(title_text="Stock Price", secondary_y=False)
    fig.update_yaxes(title_text="Revenue", secondary_y=True)
    fig.show()

make_graph(tesla_data, tesla_revenue, "Tesla")
make_graph(gme_data, gme_revenue, "GameStop")